In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

In [40]:
data = pd.read_csv("AmesHousing.tsv", delimiter='\t')

In [41]:
# Exploratory analysis
print(data.shape)
data.head()

(2930, 82)


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


Define Functions

In [42]:
def transform_features(df):
    return df

In [43]:
def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

In [44]:
def train_and_test(df):
    # Selects the first 1460 rows from data and assigns to train.
    # Selects the remaining rows from data and assigns to test
    train = df[:1460]
    test = df[1460:]
    
    # Trains a model with all numerical columns except 'SalePrice'
    # on the data frame returned from select_features()
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    
    # Tests the model on the test set and returns the RMSE value
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

In [45]:
# Apply functions to untransformed data
transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

**Feature Engineering**  
Our aim is to update transform_features() such that it:

1. removes features that we don't want to use in the model, which are those columns which have more than 5% of their values missing.
2. transforms features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc)
3. creates new features by combining other features

The transform_features() function shouldn't modify the train data frame and instead return a new one entirely. This way, we can keep using train in the experimentation cells.

In [46]:
# 1. Drop columns that have more than 5% missing values
num_missing = data.isnull().sum()

In [47]:
# Filter Series to columns containing >5% missing values
drop_missing_cols = num_missing[(num_missing > len(data)/20)].sort_values()

# Drop those columns from the data frame.
data = data.drop(drop_missing_cols.index, axis=1)

In [48]:
# Text columns: Drop any with 1 or more missing values for now.

# Series object: column name -> number of missing values
text_mv_counts = data.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

# Filter Series to columns containing *any* missing values
drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]

data = data.drop(drop_missing_cols_2.index, axis=1)

In [49]:
# Numerical columns- For columns with missing values, fill in with the most common value in that column
num_missing = data.select_dtypes(include=['int', 'float']).isnull().sum()
num_missing

Order               0
PID                 0
MS SubClass         0
Lot Area            0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Mas Vnr Area       23
BsmtFin SF 1        1
BsmtFin SF 2        1
Bsmt Unf SF         1
Total Bsmt SF       1
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Bsmt Full Bath      2
Bsmt Half Bath      2
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
TotRms AbvGrd       0
Fireplaces          0
Garage Cars         1
Garage Area         1
Wood Deck SF        0
Open Porch SF       0
Enclosed Porch      0
3Ssn Porch          0
Screen Porch        0
Pool Area           0
Misc Val            0
Mo Sold             0
Yr Sold             0
SalePrice           0
dtype: int64

In [50]:
# Treat those numeric columns which have less than 5% data missing
# as fixable
fixable_numeric_cols = num_missing[(num_missing < len(data)/20) & (num_missing > 0)].sort_values()
fixable_numeric_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [51]:
# Compute the most common value for each column in `fixable_nmeric_missing_cols`.
replacement_values_dict = data[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [52]:
# Replace missing values with these values
data = data.fillna(replacement_values_dict)

In [53]:
# Verify that every column has no missing values
data.isnull().sum().value_counts()

0    64
dtype: int64

Create new columns that determine how old the house is and how much time has passed since the last remodelling.

In [54]:
# Create new columns
data['Years Before Sale'] = data['Yr Sold'] - data['Year Built']
data['Years Since Remod'] = data['Yr Sold'] - data['Year Remod/Add']

In [55]:
# Check for negative values in these newly created columns
data[data['Years Before Sale'] < 0]

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,Years Before Sale,Years Since Remod
2180,2181,908154195,20,RL,39290,Pave,IR1,Bnk,AllPub,Inside,...,0,0,17000,10,2007,New,Partial,183850,-1,-2


In [56]:
data[data['Years Since Remod'] < 0]

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,Years Before Sale,Years Since Remod
1702,1703,528120010,60,RL,16659,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,6,2007,New,Partial,260116,0,-1
2180,2181,908154195,20,RL,39290,Pave,IR1,Bnk,AllPub,Inside,...,0,0,17000,10,2007,New,Partial,183850,-1,-2
2181,2182,908154205,60,RL,40094,Pave,IR1,Bnk,AllPub,Inside,...,0,0,0,10,2007,New,Partial,184750,0,-1


In [57]:
# Remove these rows
data = data.drop([1702, 2180, 2181], axis=0)

In [58]:
# Original year columns not needed anymore
data = data.drop(["Year Built", "Year Remod/Add"], axis = 1)

We need to drop columns that that aren't useful for building a model. We see that 'Order' and 'PID' columns are not useful for machine learning as they are simply identification numbers.

In [59]:
# Drop columns that aren't useful for model building
data = data.drop(["PID", "Order"], axis=1)

# Drop columns that leak info about the final sale
data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

Update transform_features() function

In [61]:
# We drop Sales information as well
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df

In [63]:
# Apply these functions to the data and re-calculate rms
data = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transformed_data = transform_features(data)
filtered_data = select_features(transformed_data)
rmse = train_and_test(filtered_data)

rmse

55275.36731241307

**Feature Selection**  
We'll use only numerical columns for building the model.

In [64]:
numerical_data = transformed_data.select_dtypes(include=['int', 'float'])

Create correlation heatmap matrix of the numerical features in the training data set with the target column, SalePrice

In [65]:
abs_corr_coeffs = numerical_data.corr()['SalePrice'].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2         0.006127
Misc Val             0.019273
Yr Sold              0.030358
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice 

In [66]:
# Keep columns with a correlation coefficient of larger than 0.4 
abs_corr_coeffs[abs_corr_coeffs > 0.4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [67]:
# Drop columns which have less than 0.4 correlation with SalePrice
transformed_data = transformed_data.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1)

In [68]:
transformed_data.shape

(2927, 39)

Which categorical columns to keep?

In [69]:
# Create list of column names from documentation 
# that are meant to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

1. Which columns are currently numerical but need to be encoded as categorical instead since these numbers do not have a semantic meaning?
2. Should we keep a categorical column that has hundreds of unique values (or categories)? When we do one hot encoding of this column, it will generate hundreds of columns.

In [76]:
# What are the categorical columns in our data currently? 
transform_cat_cols = []
for col in nominal_features:
    if col in transformed_data.columns:
        transform_cat_cols.append(col)

In [77]:
# How many unique values are there in each categorical column?
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()

In [78]:
uniqueness_counts

Street           2
Central Air      2
Land Contour     4
Lot Config       5
Bldg Type        5
Roof Style       6
Foundation       6
Heating          6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
Exterior 2nd    17
Neighborhood    28
dtype: int64

In [79]:
# Keep columns with atleast 10 unique values
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transformed_data = transformed_data.drop(drop_nonuniq_cols, axis=1)

In [80]:
# Select the remaining text columns and convert them to categorical
text_cols = transformed_data.select_dtypes(include=['object'])
for col in text_cols:
    transformed_data[col] = transformed_data[col].astype('category')
    
# Create dummy columns and add back to the dataframe
transformed_data = pd.concat([
    transformed_data, 
    pd.get_dummies(transformed_data.select_dtypes(include=['category']))
], axis=1)

Update select_features() function

In [81]:
def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    
    return df

**Training and testing the model**

We define a parameter k that controls the type of cross validation that occurs.

1. For k = 0, perform holdout validation:
Select the first 1460 rows and assign to train.
Select the remaining rows and assign to test.
Train on train and test on test.
Compute RMSE and return its value.

2. For k = 1, perform simple cross validation:
Shuffle the ordering of the rows in the data frame.
Select the first 1460 rows and assign to fold_one.
Select the remaining rows and assign to fold_two.
Train on fold_one and test on fold_two.
Train on fold_two and test on fold_one.
Compute the average RMSE and return its value.

3. In general, for k > 0, implement k-fold cross validation using k folds:
Perform k-fold cross validation using k folds.
Calculate the average RMSE value and return this value

In [84]:
# Define train_and_test function as described above
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    # Case 1 - Holdout validation
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    # Case 2 - Simple cross validation
    if k == 1:
        # Randomize all rows
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else: # Case 3 - K-fold cross validation
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

In [86]:
data = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transformed_data = transform_features(data)
filtered_data = select_features(transformed_data)

In [88]:
# Try with k = 4
rmse = train_and_test(filtered_data, k=4)

[35808.20591649274, 25850.491167270007, 27078.684885038965, 27819.03549818993]


In [89]:
# Try with k = 3
rmse = train_and_test(filtered_data, k=3)

[26119.42118709221, 27643.026969820654, 34543.96566554323]


In [90]:
# Try with k = 2
rmse = train_and_test(filtered_data, k=2)

[27218.442158455753, 32598.58702469266]
